In [58]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,dataset,TensorDataset
from sklearn import datasets
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [59]:

if torch.cuda.is_available():
    device=torch.device("cuda")
    print("cuda is activated!")

cuda is activated!


In [60]:
csv_path="D:/year 3/hk2/Machine Learning/testcode/media prediction and its cost.csv"
df = pd.read_csv(csv_path)
data= df[['store_sales(in millions)','store_cost(in millions)','total_children','avg_cars_at home(approx)',
          'num_children_at_home','gross_weight','net_weight','store_sqft','grocery_sqft',
            'units_per_case','SRP']].values
# data= df[['gross_weight','net_weight']]
# print(data.info())
data_output=df[['cost']].values
# sc= MinMaxScaler()
# data=sc.fit_transform(data)
# x_test=sc.transform(x_test)

In [61]:
n_sample, n_feature= data.shape
x_train, x_test, y_train, y_test = train_test_split(data,data_output,test_size=0.2,random_state=42)

In [62]:
sc= StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [63]:
# x_train=torch.from_numpy(x_train.astype(np.float32)).to(device=device)
# x_test=torch.from_numpy(x_test.astype(np.float32)).to(device=device)
# y_train=torch.from_numpy(y_train.to_numpy().astype(np.float32)).to(device=device)
# y_test=torch.from_numpy(y_test.to_numpy().astype(np.float32)).to(device=device)
# y_train=y_train.view(y_train.shape[0],1)
# y_test=y_test.view(y_test.shape[0],1)
x_train = torch.tensor(x_train, dtype=torch.float32).to(device=device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device=device).view(-1, 1)
x_test = torch.tensor(x_test, dtype=torch.float32).to(device=device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device=device).view(-1, 1)

In [64]:
batch_size = 64
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [65]:
# class linearregression (nn.Module):
#     def __init__(self,n_input_feature):
#         super(linearregression,self).__init__()
#         self.linear1=nn.Linear(n_input_feature,32,device=device)
#         self.relu1=nn.ReLU(inplace=True)
#         self.soft=nn.Softmax(dim=0)

#         self.linear2=nn.Linear(16,32,device=device)
#         self.relu2=nn.ReLU(inplace=True)

#         self.linear3=nn.Linear(32,16,device=device)
#         self.relu3=nn.ReLU(inplace=True)
        
#         self.linear4=nn.Linear(32,1,device=device)
#         self.relu4=nn.ReLU(inplace=True)
#     def forward(self, x):
#         out=self.soft(self.linear1(x))
#         # out=self.relu2(self.linear2(out))
#         # out=self.relu3(self.linear3(out))
#         return (self.linear4(out))
class linearregression(nn.Module):
    def __init__(self, input_size):
        super(linearregression, self).__init__()
        self.fc1 = nn.Linear(input_size, 64,device=device)
        self.fc2 = nn.Linear(64, 32,device=device)
        self.fc3 = nn.Linear(32, 16,device=device)
        self.fc4 = nn.Linear(16, 1,device=device)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)  # No activation for regression
        return x

In [66]:
learning_rate=0.01
model = linearregression(n_feature)
loss=nn.MSELoss()
optium=torch.optim.Adam(model.parameters(),learning_rate)


In [67]:
for epoch in range(100):
    model.train()
    l=0.0
    for batch_X, batch_y in train_loader:
        optium.zero_grad()
        y_predicted=model.forward(batch_X)
        los=loss(y_predicted,batch_y)
        los.backward()
        optium.step()
        l+=los.item()
    if (epoch+1)%10==0:
        print(f"epoch {epoch+1}: the loss = {l/len(train_loader):.5f}")
# for epoch in range(100):
#     model.train()
#     optium.zero_grad()
#     y_predicted=model(x_train)
#     l=loss(y_predicted,y_train)
#     l.backward()
#     optium.step()
#     if (epoch+1)%10==0:
#         print(f"epoch {epoch+1}: the loss = {l.item():.5f}")

epoch 10: the loss = 891.04919
epoch 20: the loss = 872.60781
epoch 30: the loss = 866.75097
epoch 40: the loss = 862.10854
epoch 50: the loss = 859.46630
epoch 60: the loss = 856.11465
epoch 70: the loss = 855.16766
epoch 80: the loss = 853.03543
epoch 90: the loss = 853.23930
epoch 100: the loss = 852.43728


In [68]:
# model.eval()
with torch.no_grad():
    y_predicted=model(x_test)
    y_predicted_rounded=y_predicted.round()
    accuracy=y_predicted_rounded.eq(y_test).sum()/float(y_test.shape[0])
    print(f"the acccuracy of the model is: {accuracy:.5f}")

the acccuracy of the model is: 0.00017
